In [441]:
import pandas as pd
import math

In [1319]:
DEADHEAD = 15
MAX_DRIVING_TIME = 480
ORIGIN = 'Origin'
DESTINATION = 'Destination'
DEPARTURE = 'Departure'
ARRIVAL = 'Arrival'

In [1320]:
file = 'service_trips.csv'
trips_df = pd.read_csv(file,index_col=0)

In [1328]:
def clac_dfs(ids):   
    for i in ids:
        for j in ids:
            valid_pair(i,j)

In [1322]:
def select_last_route(ids):
    if not ids:
        return None
   
    r_df = routing_df.loc[ids]
    t_df = trips_df.loc[ids]
    
    last_routes = r_df.loc[ (r_df[r_df.columns] == False).all(axis=1) ].index.tolist()
    if last_routes:
        return last_routes[0]
    return t_df[DEPARTURE].idxmax()
    ## here need to decide what to do when there are 2+ all False rows.

In [1407]:
def get_best_next_route(routes, last_route):
    deadheads = deadhead_df.loc[routes][last_route]
    min_deadhead = min(deadheads)
    candidates = deadheads[deadheads == min_deadhead].index.tolist()
    return trips_df.loc[candidates][DEPARTURE].idxmax()
#     return deadheads.idxmin()

In [1408]:
def get_candidates(routes, last_route):
    deadheads = deadhead_df.loc[routes][last_route]
    min_deadhead = min(deadheads)
    candidates = deadheads[deadheads == min_deadhead].index.tolist()
    return candidates
#     return trips_df.loc[candidates][DEPARTURE].idxmax()
#     return deadheads.idxmin()

In [1416]:
def get_best_from_candidates(vehicle, candidates):
    final_candidates = []
    for c in candidates:
        temp_vehicle = vehicle.copy()
        temp_vehicle.insert(0, c)
        if restricted_legal_vehicle(temp_vehicle):
            final_candidates.append(c)
    if not final_candidates:
        return None
    return trips_df.loc[final_candidates][DEPARTURE].idxmax()

In [1431]:
%%time
# random_exaple = trips_df.sample(n=50)
# ids = random_exaple.index.tolist()
# ids = [65, 78, 299, 306, 413]
# ids = [319, 67, 28, 256, 180, 159, 327, 476, 387, 383]
# ids = [1,27,28, 100]
# ids = [1,27,28]
ids = trips_df.index.tolist()

deadhead_df = pd.DataFrame(index=ids,columns=ids)
# drivingdf = pd.DataFrame(index=ids,columns=ids,data=np.zeros())
routing_df = pd.DataFrame(index=ids,columns=ids)
ids_copy = ids.copy()

clac_dfs(ids=ids)
last_route = select_last_route(ids=ids)
vehicle = []
daily_schedule = []

while last_route:
    vehicle.insert(0, last_route)
#     print('-----------')
#     print(last_route)
    ids.remove(last_route)

    routes = routing_df.loc[ids][routing_df[last_route] == True].index.tolist()
#     print(routes)
    if routes:
        candidates = get_candidates(routes=routes, last_route=last_route)
#         print(candidates)
        next_route = get_best_from_candidates(vehicle=vehicle, candidates=candidates)
#         print(next_route)
#         next_route = get_best_next_route(routes=routes, last_route=last_route)            
    if not routes or not next_route:
        daily_schedule.append(vehicle)
        vehicle = []
        next_route = select_last_route(ids=ids)

    last_route = next_route

C:\Users\sotmazgi\AppData\Local\Continuum\anaconda3\lib\site-packages\ipykernel_launcher.py:25: UserWarning: Boolean Series key will be reindexed to match DataFrame index.


Wall time: 1h 5min 7s


In [1436]:
# my answer
print(f'daily_schedule -> {valid_daily_schedule(daily_schedule)}')
# print(daily_schedule)
print(f'vehicles: {len(daily_schedule)}')
print(f'opex: {get_opex(daily_schedule)}')
print(f'deadheads: {get_opex(daily_schedule)/15}')
print('-----vehicle_driving-----')
for v in daily_schedule:
    if get_vehicle_driving_time(v) > MAX_DRIVING_TIME:
        print(v)

daily_schedule -> True
vehicles: 89
opex: 2730
deadheads: 182.0
-----vehicle_driving-----


In [1433]:
trips_df.shape

(508, 4)

In [1430]:
display(trips_df.loc[ids_copy])
# display(routing_df)
# display(deadhead_df)

,Origin,Destination,Departure,Arrival
Id,,,,
319,10,1,930,960
67,1,1,1035,1095
28,1,1,840,900
256,1,1,920,1070
180,12,5,677,737
159,4,3,720,780
327,10,1,1170,1200
476,8,8,405,495
387,6,11,540,570


In [1399]:
# show all the options
# for partition in partitions(set(ids_copy)):
#     if not valid_daily_schedule(partition):
#         continue
#     print(partition)
#     print(get_opex(partition))

In [447]:
from itertools import tee
def pairwise(iterable):
    "s -> (s0,s1), (s1,s2), (s2, s3), ..."
    a, b = tee(iterable)
    next(b, None)
    return zip(a, b)

In [448]:
def get_service_trip(id):
    global trips_df
    return trips_df.loc[id]

In [449]:
def get_driving_time(s1,s2):
    return s1-s2
#     return trip[ARRIVAL] - trip[DEPARTURE]

In [498]:
def get_deadhead(id_1, id_2):
    if id_1 == id_2:
        return 0
    service_trip_1 = get_service_trip(id_1)
    service_trip_2 = get_service_trip(id_2)
    return DEADHEAD if service_trip_1[DESTINATION] != service_trip_2[ORIGIN] else 0

def get_deadhead_duration(vehicle):
    if len(vehicle) == 1:
        return 2*DEADHEAD
    return 2*DEADHEAD + sum( [get_deadhead(curr_trip_id,next_trip_id) 
                              for curr_trip_id, next_trip_id in pairwise(vehicle)])

In [1362]:
def valid_pair(id_1, id_2):
    service_trip_1 = get_service_trip(id_1)
    service_trip_2 = get_service_trip(id_2)
    if id_1 == id_2:
        routing_df.loc[id_1,id_2] = False
        deadhead_df.loc[id_1,id_2] = -1
        return
    
    deadhead = get_deadhead(id_1,id_2)    
    if service_trip_2[DEPARTURE] < (service_trip_1[ARRIVAL] + deadhead):
        routing_df.loc[id_1,id_2] = False
        deadhead_df.loc[id_1,id_2] = -1
        return
    deadhead_df.loc[id_1,id_2] = deadhead
    routing_df.loc[id_1,id_2] = True

In [1281]:
def valid_vehicle(vehicle):
    for curr_trip_id, next_trip_id in pairwise(vehicle):
        curr_service_trip = get_service_trip(curr_trip_id)
        if curr_service_trip[ARRIVAL] - curr_service_trip[DEPARTURE] < 0:
            return False
        
        next_service_trip = get_service_trip(next_trip_id)
        if next_service_trip[ARRIVAL] - next_service_trip[DEPARTURE] < 0:
            return False
        
        deadhead = get_deadhead(curr_trip_id,next_trip_id)
        if next_service_trip[DEPARTURE] < (curr_service_trip[ARRIVAL] + deadhead):
            return False
       
    return True

def valid_daily_schedule(daily_schedule):
    for v in daily_schedule:
        if not valid_vehicle(v):
            return False
    return True

In [453]:
def get_vehicle_driving_time(vehicle):
    vehicle_duration = 0
    deadhead_duration = get_deadhead_duration(vehicle)
    for trip_id in vehicle:
        service_trip = get_service_trip(trip_id)
        vehicle_duration += service_trip[ARRIVAL] - service_trip[DEPARTURE]
    return vehicle_duration + deadhead_duration

In [454]:
def restricted_legal_vehicle(vehicle):
    if not valid_vehicle(vehicle=vehicle):
        return False
    return get_vehicle_driving_time(vehicle=vehicle) <= MAX_DRIVING_TIME

In [455]:
def get_opex(vehicle_list):
    return sum( [get_deadhead_duration(v) for v in vehicle_list] )

In [457]:
trips_df[DIFF] =get_driving_time(trips_df[ARRIVAL],trips_df[DEPARTURE])

In [458]:
valid_vehicle(example)

True

In [459]:
get_deadhead_duration(example)

45

In [460]:
get_vehicle_driving_time(example)

225

In [461]:
restricted_legal_vehicle(example)

True

In [462]:
from itertools import chain, combinations

def powerset(iterable):
    "powerset([1,2,3]) --> () (1,) (2,) (3,) (1,2) (1,3) (2,3) (1,2,3)"
    s = list(iterable)
    return chain.from_iterable(combinations(s, r) for r in range(len(s)+1))

In [48]:
%%time
for p in powerset([1,27,28]):
    print(p)

()
(1,)
(27,)
(28,)
(1, 27)
(1, 28)
(27, 28)
(1, 27, 28)
Wall time: 2 ms


In [74]:
def partitions(A):
    if not A:
        yield []
    else:
        a, *R = A
        for partition in partitions(R):
            yield partition + [[a]]
            for i, subset in enumerate(partition):
                yield partition[:i] + [[a] + subset] + partition[i+1:]

In [75]:
for partition in partitions({1, 27, 28}):
    print(partition)

[[28], [27], [1]]
[[1, 28], [27]]
[[28], [1, 27]]
[[27, 28], [1]]
[[1, 27, 28]]


In [143]:
daily_schedule = []
opex = []
for partition in partitions({1, 2, 3, 4, 5, 6, 7, 8, 9, 10}):
    for p in partition:
        if not restricted_legal_vehicle(p):
            continue
    daily_schedule.append(partition)
    opex.append(get_opex(partition))

In [112]:
import numpy as np

In [145]:
daily_schedule[np.argmin(opex)]

[[1, 2, 3, 4, 5, 6, 7]]

In [98]:
v = []
for i in trips_df.index:
    v.append(i)
    if not restricted_legal_vehicle(v):
        print(v)
        break

[1, 2, 3, 4, 5, 6, 7, 8]


In [99]:
valid_vehicle(v)

True

In [100]:
restricted_legal_vehicle(v)

False

In [212]:
ids = [1, 27, 28, 100]

In [170]:
origin_stations = set(trips_df[ORIGIN].unique())
destination_stations = set(trips_df[DESTINATION].unique())
origin_stations.union(destination_stations)

{1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12}

In [165]:
stations = trips_df[DESTINATION].unique()
stations

array([ 9,  2,  1,  4,  3, 10,  7, 12,  5,  6, 11,  8], dtype=int64)

In [326]:
import numpy as np

In [327]:
df = pd.DataFrame(index=ids,columns=ids,data=0)

In [328]:
for i in range(len(df.index)):
    for j in range(len(df.columns)):
        #print(i,j)

        if i == j:
            df.iloc[i,j] = 0
        else:
            id_1 = ids[j-1]
            id_2 = ids[j]
            if valid_pair(id_1,id_2):
                df.iloc[i,j] = get_deadhead(id_1,id_2)
            else:
                df.iloc[i,j] = -1
#         print(id_1,id_2)
#         print()
    

In [418]:
ids = [1, 27, 28,100]
df = trips_df.loc[ids]

In [421]:
df['sort'] = -1
df

,Origin,Destination,Departure,Arrival,sort
Id,,,,,
1,2,9,305,365,-1
27,1,1,780,840,-1
28,1,1,840,900,-1
100,10,1,480,510,-1


In [463]:
trips_df[0:19]

,Origin,Destination,Departure,Arrival,Diff
Id,,,,,
1,2,9,305,365,60
2,9,2,365,425,60
3,2,9,425,485,60
4,9,2,485,545,60
5,2,9,545,605,60
6,9,2,605,665,60
7,2,9,665,725,60
8,9,2,725,785,60
9,2,9,785,845,60


In [414]:
for i in ids:
    for j in ids:  
        df.loc[i,j] = valid_pair(i,j)
df       

,1,27,28,100
1,True,True,True,True
27,False,True,True,False
28,False,False,True,False
100,False,True,True,True
